In [7]:
library(readxl)
library(data.table)

In [8]:
set.seed(1234)

# (a)

I think $n = 1000$ is a good number for the size of `demos`.

In [9]:
n = 1000

In [10]:
birth_dt = sample(seq(as.Date('1940/01/01'), as.Date('2019/12/31'), by="day"), n)

(Note: this is definitely not the same as the date range you asked for, but bear with me, I'll explain at the very end why I made this decision).

I suppose I could use numbers between 1 and $n$ as the ids, but that feels less like how a real id system. Of course, the ids aren't a numerical variable more than a categorical one so it doesn't matter too much, but I think using something that feels more like real ids would also be a bit more fun.

In [11]:
counties = read.csv("Alphabetical-List-Of-California-Counties.csv")[,1]

Then we create the basic scaffolding of our dataset. This involves counting up id's from 1 to $n$, and using our birth date samples, and then randomly picking a gender and county.

In [12]:
demo = data.table(id=1:n,
           birth_dt=birth_dt,
           gender=sample(c(1L,2L), size=n, replace=TRUE),
           county=sample(counties, size=n, replace=TRUE)
          )

setkey(demo, id)

In [13]:
head(demo)

id,birth_dt,gender,county
<int>,<date>,<int>,<chr>
1,1960-05-26,2,Mendocino
2,1961-12-11,2,Inyo
3,1959-08-10,1,El Dorado
4,1962-02-19,2,Trinity
5,2004-10-03,1,Mariposa
6,1965-03-05,1,Placer


## (iv)

Right, so now we need create `Medical_visit`. My plan is to just start with creating a vector of ids, paying attention to the fact that there can be multiple of the same id on different days.

My original plan was to sample `n` entries from a vector that looked like (1,2,3,4,5) to get the number of times each id should appear, and weigh the choices with 80% for a 1 (80% chance of only one visit), then evenly split the remaining 20%. However, this of course doesn't guarantee *exactly* 20% of ids have only one visit. I know that the numbers probably don't have to be that precise, but I thought it would also be more intuitive.

So as for the strategy, we can just create a vector of $.70 \cdot n$ ones, and then another vector of $.2 \cdot n$ non-ones sampled from (2,3,4,5)

In [14]:
one_visit = rep(1, .7 * n)

no_visit = rep(0, .1 * n)

So now for the multiple visits, I decided to have a bit of fun. I remembered in one of my probability classes that we learned about the Poisson distribution, which can represent the amount of time before an event that happens randomly occurs again. This can be a mini-simulation for the random variable of when people need to visit the hospital.

Using this idea, we can use the distribution, and then imagine in this normalized scenario that getting a number of 3.17 for example means you needed to go once with a "frequency" of 3.17, or 3.17 times this time period.

These are ones for multiple visits, so due to truncation and to fit the bill we just need to add 2.

At the end of the day, I could have just sampled uniformly, so this is just me having a bit of fun.

In [15]:
mult_visits = trunc(rpois(.2 * n, lambda = .3)) / 1 + 2
mult_visits

[1] 2 2 2 2 2 2 2 3 3 2 2 2 2 2 2 2 2 2 2 3 2 3 5 4 3 3 2 2 4 2 2 2 2 2 3 2 2
 [38] 2 3 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 2 2 2 2 2 2 2 2 3 2 3 3 2 2 2 2 2 2 2 3
 [75] 2 2 2 2 2 3 2 2 2 2 3 2 3 2 2 3 2 2 3 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
[112] 2 2 2 2 2 3 2 3 2 3 2 2 2 2 2 3 2 2 2 3 2 2 3 2 2 2 2 2 3 2 2 2 2 2 3 2 2
[149] 2 3 2 2 2 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 3 2 2 3 3 2 2 3 2 2 2 2 2 2
[186] 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2

In [16]:
visits_num = sample(c(one_visit, no_visit, mult_visits))

In [17]:
length(visits_num)
sum(visits_num)

[1] 1000

[1] 1145

So this represents that we have 1000 total ids, and 1145 total visits after accounting for people who didn't visit at all, and people who visit more than once.

Then was we planned earlier, we can finally use `rep` to get one row (one visit) repeating a number of times based on `visits_num`.

In [29]:
Medical_visit = demo[rep(seq_len(nrow(demo)), visits_num), ]

In [30]:
Medical_visit[, service_date := sample(seq(max(as.Date('2018/01/01'), birth_dt), as.Date("2019/12/31"), by="day"), size = 1, replace = TRUE), by=1:nrow(Medical_visit)]
Medical_visit[, service_num  := sample(seq(from=1, to=20), size = .N, replace = FALSE), by=.(id,service_date)]

In [31]:
head(Medical_visit)

id,birth_dt,gender,county,service_date,service_num
<int>,<date>,<int>,<chr>,<date>,<int>
1,1960-05-26,2,Mendocino,2019-12-27,20
2,1961-12-11,2,Inyo,2019-03-06,18
3,1959-08-10,1,El Dorado,2019-12-04,6
3,1959-08-10,1,El Dorado,2018-01-21,13
4,1962-02-19,2,Trinity,2019-09-27,11
4,1962-02-19,2,Trinity,2019-02-22,12


# (b)

The logic is very simple to implement as an "and", and then we just use data.table's nice subsetting with `i`.

In [15]:
Medical_visit[gender==2 & county=="Los Angeles"]

id,birth_dt,gender,county,service_date,service_num
<int>,<date>,<int>,<chr>,<date>,<int>
81,2011-07-08,2,Los Angeles,2018-07-28,8
184,1944-09-20,2,Los Angeles,2018-01-14,19
358,1956-08-18,2,Los Angeles,2019-11-30,17
483,1959-12-21,2,Los Angeles,2019-01-06,18
690,1973-11-01,2,Los Angeles,2018-03-02,3
766,1987-09-30,2,Los Angeles,2019-10-14,14
786,1998-10-08,2,Los Angeles,2019-12-10,1
792,2015-07-12,2,Los Angeles,2018-02-14,18
805,1947-06-28,2,Los Angeles,2019-10-12,5


# (c)

This is once again very simple using data.table subsetting with `i`.

In [16]:
head(Medical_visit[service_date <= as.Date("2018-05-14")])

id,birth_dt,gender,county,service_date,service_num
<int>,<date>,<int>,<chr>,<date>,<int>
3,1959-08-10,1,El Dorado,2018-01-21,13
5,2004-10-03,1,Mariposa,2018-01-14,19
17,1991-05-10,1,Tuolumne,2018-01-11,19
19,1973-07-03,1,Merced,2018-03-23,11
33,1941-04-23,1,Amador,2018-04-05,15
34,1972-06-29,1,Del Norte,2018-05-05,10


# (d)

Actually a really annoying bug I was running into with my method here is that I had something like

```
service_date[service_num == max(service_num)]
```

as the `j` for my data.table operation, but I did forget that fact that there could be a tie for the service number, meaning I'd potentially get multiple dates per id, which is not what we want. So I just realized I had 1 sample from this value which had a potentially greater than 1 length. This also nicely handles the random tiebreaking we're asked for.

In [17]:
Medical_visit[, max_svc_date:=sample(service_date[service_num == max(service_num)], size=1), by=id]

In [18]:
head(Medical_visit)

id,birth_dt,gender,county,service_date,service_num,max_svc_date
<int>,<date>,<int>,<chr>,<date>,<int>,<date>
1,1960-05-26,2,Mendocino,2019-12-27,20,2019-12-27
2,1961-12-11,2,Inyo,2019-03-06,18,2019-03-06
3,1959-08-10,1,El Dorado,2019-12-04,6,2018-01-21
3,1959-08-10,1,El Dorado,2018-01-21,13,2018-01-21
4,1962-02-19,2,Trinity,2019-09-27,11,2018-11-02
4,1962-02-19,2,Trinity,2019-02-22,12,2018-11-02


## (e)

Just so that we can see this work here, I want to wrap the creation of `demo` and `medical_visit_10` all into a function. Of course, one concern about how we're going to merge everything in the end is what happens if we have id collisions? To solve this problem all we need to do is have our function be able to take a user-specified range of ids with which to create a dataset. Then we can just supply 10 disjoint ranges of ids, one for each of the 10 resulting pairs of dataframes.

In [19]:
create_demo_visits_pair = function(id_range) {
    n = length(id_range)
    counties = read.csv("Alphabetical-List-Of-California-Counties.csv")[,1]
    birth_dt = sample(seq(as.Date('1940/01/01'), as.Date('2019/12/31'), by="day"), n)
    demo = data.table(id=id_range,
           birth_dt=birth_dt,
           gender=sample(c(1L,2L), size=n, replace=TRUE),
           county=sample(counties, size=n, replace=TRUE)
          )
    
    setkey(demo, id)
    
    one_visit = rep(1, .7 * n)
    no_visit = rep(0, .1 * n)
    
    mult_visits = trunc(rpois(.2 * n, lambda = .3)) / 1 + 2
    
    visits_num = sample(c(one_visit, no_visit, mult_visits))
    
    Medical_visit = data.table(id=rep(demo[, id], visits_num))
    setkey(Medical_visit,id)
    
    Medical_visit = Medical_visit[demo, nomatch=0]
    
    Medical_visit[, service_date := sample(seq(max(as.Date('2018/01/01'), birth_dt), as.Date("2019/12/31"), by="day"), size = 1, replace = TRUE), by=1:nrow(Medical_visit)]
    Medical_visit[, service_num := sample(seq(from=1, to=20), size = .N, replace = FALSE), by=.(id,service_date)]
    Medical_visit[, max_svc_date:=sample(service_date[service_num == max(service_num)], size=1), by=id]
    return(list(demo, Medical_visit))
}

Wrapped as a function, we can now loop 10 times really easily to simulate those 10 tuples of datasets.

Again, to avoid clashing with ids, we pass a disjoint range of ids to the function to use each time.

In [20]:
df_list = list()

for(i in seq(10)) {
    Medical_visit = create_demo_visits_pair((1000*(i-1) + 1):(1000*i))[[2]]
    girls_in_LA = Medical_visit[gender==2][county=="Los Angeles"]
    df_list[[i]] = girls_in_LA
}

combined = do.call("rbind", df_list)

In [21]:
head(combined, 10)

id,birth_dt,gender,county,service_date,service_num,max_svc_date
<int>,<date>,<int>,<chr>,<date>,<int>,<date>
36,1994-03-11,2,Los Angeles,2018-12-24,10,2018-12-24
54,1943-03-24,2,Los Angeles,2019-04-15,18,2019-04-15
246,2011-02-21,2,Los Angeles,2018-09-17,4,2018-09-17
383,1951-07-19,2,Los Angeles,2019-08-13,15,2018-01-21
383,1951-07-19,2,Los Angeles,2018-01-21,18,2018-01-21
469,1968-08-09,2,Los Angeles,2018-11-28,10,2019-05-03
469,1968-08-09,2,Los Angeles,2019-05-03,12,2019-05-03
509,2010-10-04,2,Los Angeles,2019-04-17,4,2019-04-17
781,1941-03-06,2,Los Angeles,2019-09-15,8,2019-09-15


## (f)

At this point, it becomes apparent that there's another piece of information I was missing: the 1`birth_dt` can be missing. So we have to update our previous work to account for this.

But this is as simple as randomly picking indices to be thrown out. I'll randomly choose for 5% to be thrown out, and just for fun, let's use a uniform distribution to do this.

In [22]:
num_no_birth = 0.05 * n
birth_dt = sample(seq(as.Date('1940/01/01'), as.Date('2021/12/31'), by="day"), n)
inds = round ( runif(num_no_birth, 1, length(birth_dt)) ) 

birth_dt[inds] = NA

In [23]:
create_demo_visits_pair = function(id_range) {
    n = length(id_range)
    counties = read.csv("Alphabetical-List-Of-California-Counties.csv")[,1]
    
    num_no_birth = 0.05 * n                                                           # new
    birth_dt = sample(seq(as.Date('1940/01/01'), as.Date('2019/12/31'), by="day"), n) # new
    inds = round ( runif(num_no_birth, 1, length(birth_dt)) )                         # new

    birth_dt[inds] = NA                                                              # new
    demo = data.table(id=id_range,
           birth_dt=birth_dt,
           gender=sample(c(1L,2L), size=n, replace=TRUE),
           county=sample(counties, size=n, replace=TRUE)
          )
    
    setkey(demo, id)
    
    one_visit = rep(1, .7 * n)
    no_visit = rep(0, .1 * n)
    
    mult_visits = trunc(rpois(.2 * n, lambda = .3)) / 1 + 2
    
    visits_num = sample(c(one_visit, no_visit, mult_visits))
    
    Medical_visit = data.table(id=rep(demo[, id], visits_num))
    setkey(Medical_visit,id)
    
    Medical_visit = Medical_visit[demo, nomatch=0]
    
    
    Medical_visit[, service_date := sample(seq(max(as.Date('2018/01/01'), birth_dt,na.rm=T), as.Date("2019/12/31"), by="day"), size = 1, replace = TRUE), by=1:nrow(Medical_visit)]
    Medical_visit[, service_num := sample(seq(from=1, to=20), size = .N, replace = FALSE), by=.(id,service_date)]
    Medical_visit[, max_svc_date:=sample(service_date[service_num == max(service_num)], size=1), by=id]
    
    return(list(demo, Medical_visit))
}

In [24]:
pair = create_demo_visits_pair(1:1000)
demo = pair[[1]]
Medical_visit = pair[[2]]

The age then is just the difference between the service date and birth date, and I'll convert to years so it looks a little nicer.

In [25]:
Medical_visit[, age_by_svc:=round(as.numeric(difftime(service_date,birth_dt,units="weeks"))/52.25, 2)]

In [26]:
head(Medical_visit, n=10)

id,birth_dt,gender,county,service_date,service_num,max_svc_date,age_by_svc
<int>,<date>,<int>,<chr>,<date>,<int>,<date>,<dbl>
1,1954-02-04,1,Mariposa,2019-01-18,7,2019-01-18,64.86
2,1996-07-28,1,Yuba,2019-01-25,20,2019-01-25,22.46
3,1964-04-18,2,Plumas,2018-07-22,16,2018-07-22,54.18
4,1980-09-15,2,Tuolumne,2019-10-28,10,2019-10-28,39.06
5,NA,1,Kern,2019-11-12,11,2019-11-12,NA
6,1999-12-16,2,Contra Costa,2019-03-26,15,2019-03-26,19.25
7,1940-09-25,2,Glenn,2018-02-19,20,2018-02-19,77.30
8,1980-08-06,2,Santa Barbara,2019-09-30,12,2019-09-30,39.10
9,1999-06-23,2,San Bernardino,2019-04-09,4,2019-04-09,19.77


# (g)

We just need ones that have a county record, and have the correct date range.

In [27]:
Medical_visit2018 = Medical_visit[!is.na(county)
                                  & service_date >= as.Date("2018-01-01")
                                  & service_date <= as.Date("2018-12-31")]

In [28]:
head(Medical_visit2018, n=20)

id,birth_dt,gender,county,service_date,service_num,max_svc_date,age_by_svc
<int>,<date>,<int>,<chr>,<date>,<int>,<date>,<dbl>
3,1964-04-18,2,Plumas,2018-07-22,16,2018-07-22,54.18
7,1940-09-25,2,Glenn,2018-02-19,20,2018-02-19,77.30
15,2004-09-04,1,Alpine,2018-02-25,19,2018-02-25,13.46
17,2007-09-27,2,Tehama,2018-12-19,19,2018-12-19,11.21
23,1973-06-24,2,Monterey,2018-12-03,15,2018-12-03,45.38
26,1962-07-01,2,Mariposa,2018-11-22,14,2018-11-22,56.32
29,1998-11-13,2,San Bernardino,2018-11-29,11,2018-11-29,20.02
31,2002-01-13,2,Marin,2018-02-10,2,2018-02-10,16.05
32,1941-01-15,1,Inyo,2018-03-22,1,2018-03-22,77.07


We can once again use an apply to get statistics summaries from each county, with a specialized summary function to account for the fact that some counties might actually have no NA values. Otherwise the length of the result of summary can vary and then data.table gets confused.

In [29]:
my_summary = function(v){
  if(!any(is.na(v))){
    res = c(summary(v),"NA's"=0)
  } else{
    res = summary(v)
  }
  return(res)
}

Age_distri = Medical_visit2018[, as.list(my_summary(age_by_svc)), by = county]
Age_distri = Age_distri[, `NA's`:=NULL]

setcolorder(Age_distri, c("county","Min.", "1st Qu.", "Median", "3rd Qu.", "Max.", "Mean"))

In [30]:
head(Age_distri)

county,Min.,1st Qu.,Median,3rd Qu.,Max.,Mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Plumas,0.00,10.540,50.130,56.875,74.02,38.59333
Glenn,8.71,32.430,45.400,48.280,77.30,40.89667
Alpine,7.47,16.840,39.470,71.070,77.91,41.46364
Tehama,11.21,32.805,48.345,58.130,75.58,45.16000
Monterey,10.18,27.970,33.780,38.520,67.31,35.32556
Mariposa,43.39,52.030,56.320,58.560,58.84,53.82800


By far the part of the assignment that took the longest figuring out all the age stuff correctly, like during simulations it'd be very likely you'd get a good amount of people who had service dates before they were born. I was torn between a couple options:
1. Programming a method that would change any birthdays based on the service date
2. Programming a method that would change any service days based on the birth date
3. Programming a method that at the very beginning only choose dates after the birth date
4. Delete all rows with invalid birthdays/service days

I figured out a way to do all of these, but I realized that they were probably all flawed in some fundamental way. For example, changing dates after adding so much information was probably not a good practice, and option 3 is actually more logically flawed: there are some birthdates that happens even completely after the given service range. And option 4 would change the size of the dataset, and I wanted to have all rows stick throughout the entire process. 

Perhaps I could have had a fancy method as well to have NA information on those entries, but at that point, what's the difference between deleting those rows, i.e. not having them around in the first place. 

For those reasons, you've already seen what I ended up going with: I changed the range of valid birthdays so that we only include people that were born before the last recorded service day. This was probably the most obvious option, but I was just initially a bit afraid of straying too much from the specifications, but I just imagined if I were really working on a simulation like this for my own research, I probably wouldn't hesitate to do something like this.